# Assignment

In [1]:
# Import 

import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env import *


size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [2]:
## INITIALISE THE ENVIRONMENT

env=Chess_Env(size_board)

In [4]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S,X,allowed_a=env.Initialise_game()                       # INTIALISE GAME

print(S)                                                  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ',env.check)                                # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))    # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO


for i in range(5):
    
    a,_=np.where(allowed_a==1)                  # FIND WHAT THE ALLOWED ACTIONS ARE
    a_agent=np.random.permutation(a)[0]         # MAKE A RANDOM ACTION
    print("action",a_agent)
    S,X,allowed_a,R,Done=env.OneStep(a_agent)   # UPDATE THE ENVIRONMENT
    print("allowed_a",np.shape(allowed_a),allowed_a)
    
    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print("S",S)
    print(R,'', Done)
    print('check? ',env.check)
    print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))
    
    
    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break



[[0 0 0 3]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 2 0]]
check?  0
dofk2  1
action 6
allowed_a (32, 1) [[0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]]

S [[0 0 3 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 1 0 2]]
0  0
check?  0
dofk2  2
action 25
allowed_a (32, 1) [[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]

S [[0 3 0 0]
 [0 0 0 0]
 [0 1 0 0]
 [0 0 0 2]]
0  0
check?  0
dofk2  1
action 26
allowed_a (32, 1) [[0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]]

S [[3 0 0 0]
 [0 0 0 0]
 [0 0 1 0]
 [0 0 0 2]]
0  0
check?  0
dofk2  2
action 9
allowed_a (32, 1) [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [

In [5]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S,X,allowed_a=env.Initialise_game()
N_episodes=1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):
    
    S,X,allowed_a=env.Initialise_game()     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        
        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 
        
        a,_=np.where(allowed_a==1)
        a_agent=np.random.permutation(a)[0]

        S,X,allowed_a,R,Done=env.OneStep(a_agent)
        print("R",R)
        
        
        if Done:
            
            R_save_random[n]=np.copy(R)
            N_moves_save_random[n]=np.copy(i)

            break

        i=i+1                               # UPDATE THE COUNTER



# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW 
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:',np.mean(R_save_random),'Number of steps: ',np.mean(N_moves_save_random))



R 0
R 0
R 0
R 0
R 0
R 1
R 0
R 0.0
R 0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0.0
R 0.0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 1
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 1
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 1
R 0.0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 

R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 1
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 1
R 0.0
R 0.0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0


R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 1
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 1
R 0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 1
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 1
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 1
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0.0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 1
R 

R 0.0
R 0
R 0
R 0
R 0
R 0
R 1
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0.0
R 1
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 1
R 0
R 0
R 0.0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0
R 0.0
R 0
R 0
R 0
R 0
R 0.0
R 0.0
R 1
R 0
R 0
R 0
R 0
R 0
R 0


**Define Adam**

In [6]:
## Let's define Adam, see notebook Adam's playgroud for more information

class Adam:

    def __init__(self, Params, beta1):
        
        N_dim=np.shape(np.shape(Params))[0] # It finds out if the parameters given are in a vector (N_dim=1) or a matrix (N_dim=2)
        
        # INITIALISATION OF THE MOMENTUMS
        if N_dim==1:
               
            self.N1=np.shape(Params)[0]
            
            self.mt=np.zeros([self.N1])
            self.vt=np.zeros([self.N1])
        
        if N_dim==2:
            
            self.N1=np.shape(Params)[0]
            self.N2=np.shape(Params)[1]
        
            self.mt=np.zeros([self.N1,self.N2])
            self.vt=np.zeros([self.N1,self.N2])
        
        # HYPERPARAMETERS OF ADAM
        self.beta1=beta1
        self.beta2=0.999
        
        self.epsilon=10**(-8)
        
        # COUNTER OF THE TRAINING PROCESS
        self.counter=0
        
        
    def Compute(self,Grads):
                
        self.counter=self.counter+1
        
        self.mt=self.beta1*self.mt+(1-self.beta1)*Grads
        
        self.vt=self.beta2*self.vt+(1-self.beta2)*Grads**2
        
        mt_n=self.mt/(1-self.beta1**self.counter)
        vt_n=self.vt/(1-self.beta2**self.counter)
        
        New_grads=mt_n/(np.sqrt(vt_n)+self.epsilon)
        
        return New_grads

beta1=0.9 # First order momentum for Adam

In [7]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 

import numpy.matlib 
S,X,allowed_a=env.Initialise_game()
N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
print(np.shape(allowed_a))
N_in=np.shape(X)[0]    ## INPUT SIZE
N_h=200                ## NUMBER OF HIDDEN NODES


## INITALISE YOUR NEURAL NETWORK...


# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.00005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE

N_episodes = 5000 # THE NUMBER OF GAMES TO BE PLAYED 

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])
N_moves_save = np.zeros([N_episodes, 1])

#weight initialise
#W1 = np.random.randn(N_in, N_h)/1000
#W2 = np.random.randn(N_h,N_a)/1000

W1 = np.random.uniform(0,1,(N_h, N_in))
W2 = np.random.uniform(0,1,(N_a, N_h))

# The following normalises the random weights so that the sum of each row =1
W1 = np.divide(W1,np.matlib.repmat(np.sum(W1,1)[:,None],1,N_in))
W2 = np.divide(W2,np.matlib.repmat(np.sum(W2,1)[:,None],1,N_h))

bias_W1 = np.zeros((N_h,))
bias_W2 = np.zeros((N_a,))

print("W",W1,W2)
print("bias",np.shape(bias_W1),bias_W1,bias_W2)
x1=np.dot(W1,X)+bias_W1
print("X",np.shape(X),np.shape(x1))
print(np.shape(np.dot(W2,x1)+bias_W2))

(32, 1)
W [[2.13506176e-02 4.51935715e-03 1.94089239e-02 ... 3.06468915e-02
  1.44139125e-02 1.83069973e-02]
 [1.63582659e-02 2.70772340e-02 2.86397356e-02 ... 2.62543162e-02
  2.66902766e-02 8.61030253e-03]
 [2.81448538e-02 8.16575595e-03 1.32542993e-02 ... 8.03674265e-03
  2.53056479e-02 2.16033847e-02]
 ...
 [4.58998105e-03 2.61329556e-02 2.24203373e-02 ... 1.58550502e-02
  8.99250100e-04 1.05720362e-02]
 [5.00549152e-04 3.21073773e-02 3.21132166e-02 ... 2.25514918e-03
  8.65587411e-03 2.00569387e-02]
 [2.70135440e-03 6.18429722e-03 7.52001938e-06 ... 5.15923412e-04
  3.04653765e-02 2.42775669e-02]] [[0.00014664 0.00664128 0.0085849  ... 0.0028705  0.0076238  0.00159245]
 [0.00435654 0.00337849 0.00941669 ... 0.00527687 0.00423381 0.00792661]
 [0.00913039 0.0004996  0.00762687 ... 0.00576101 0.00785791 0.00930523]
 ...
 [0.00321091 0.00443095 0.00536833 ... 0.00977062 0.00731756 0.00737974]
 [0.00075235 0.00843487 0.00055227 ... 0.00734128 0.0096062  0.00573004]
 [0.00250748 0.00657

In [8]:
def EpsilonGreedy_Policy(Qvalues, epsilon):
    
    N_a=np.shape(Qvalues)[0]

    rand_value=np.random.uniform(0,1)

    rand_a=rand_value<epsilon
    a=np.where(allowed_a==1)[0]
    Qvalues_allowed=np.copy(Qvalues[a])
    if rand_a==True:
        
        a_agent=np.random.permutation(a)[0]

    else:
        '''
        sort_a=np.argsort(Qvalues)[::-1]
        while a_item in sort_a:
            if a_item in a:
                return a_item
        '''
        a1=np.argmax(Qvalues_allowed)
        a_agent=np.copy(a[a1])
            
    return a_agent

In [9]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...

for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS
    
    S,X,allowed_a=env.Initialise_game()      ## INITIALISE GAME
    print("N_episodes",n)                    ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING
    
    
    while Done==0:                           ## START THE EPISODE
        
        
        ## THIS IS A RANDOM AGENT, CHANGE IT to SARSA with value function approximation
        
        #a,_=np.where(allowed_a==1)
        #a_agent=np.random.permutation(a)[0]
        h1 = np.dot(W1,X)+bias_W1

        # Apply the sigmoid function
        x1 = 1/(1+np.exp(-h1))
        
        Qvalues=np.dot(W2,x1)+bias_W2
        
        #print("Qvalues",np.shape(Qvalues),Qvalues)
                
        #S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        a_agent = EpsilonGreedy_Policy(Qvalues, epsilon_f)
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
       # print("S",S,"action",a_agent,R,Done)
        
        ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
        if Done==1:
            print("Done",Done)
            # Compute the delta
            #delta=R-Qvalues[a]
            
            # Update the Qvalues
            delta=R-Qvalues[a_agent]
                
           
            
            W2[a_agent,:]=W2[a_agent,:]+eta*delta*x1.T
            bias_W2[a_agent] = bias_W2[a_agent]+eta*delta
            
            W1=W1+eta*delta*X.T
            bias_W1 = bias_W1+eta*delta
            #b1[a_agent,:]=b1[a_agent,:]+eta*delta*X
            #b2=b2+eta*delta*h1
            #save the R_save,N_moves_save
            R_save[n]=np.copy(R)
            N_moves_save[n]=np.copy(i)
            
            
            break
        
        # IF THE EPISODE IS NOT OVER...
        h2 = np.dot(W1,X_next)+bias_W1

        # Apply the sigmoid function
        x2 = 1/(1+np.exp(-h2))
        
        Qvalues2=np.dot(W2,x2)+bias_W2
        a_pri = EpsilonGreedy_Policy(Qvalues2, epsilon_f)
        
        # Compute the delta
        delta=R+gamma*Qvalues2[a_pri]-Qvalues[a_agent]
        
        # Update the weights
        #print(np.shape(W2[a_agent,:]),np.shape(delta),np.shape(Qvalues2[a_pri]))
        W2[a_agent,:]=W2[a_agent,:]+eta*delta*x1.T
        bias_W2[a_agent] = bias_W2[a_agent]+eta*delta
            
        W1=W1+eta*delta*X.T
        bias_W1 = bias_W1+eta*delta
            
        # NEXT STATE AND CO. BECOME ACTUAL STATE...     
        S=np.copy(S_next)
        X=np.copy(X_next)
        allowed_a=np.copy(allowed_a_next)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
        print("action i",i)
        
        
    
    

N_episodes 0
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
Done 1
N_episodes 1
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
Done 1
N_episodes 2
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
Done 1
N_episodes 3
Done 1
N_episodes 4
Done 1
N_episodes 5
action i 2
action i 3
Done 1
N_episodes 6
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
Done 1
N_episodes 7
action i 2
action i 3
action i 4
action i

action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
Done 1
N_episodes 76
Done 1
N_episodes 77
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
Done 1
N_episodes 78
action i 2
action i 3
Done 1
N_episodes 79
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
Done 1
N_episodes 80
Done 1
N_episodes 81
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
actio

action i 19
action i 20
action i 21
action i 22
Done 1
N_episodes 126
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
Done 1
N_episodes 127
action i 2
action i 3
action i 4
Done 1
N_episodes 128
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 129
Done 1
N_episodes 130
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 131
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
Done 1
N_episodes 132
Done 1
N_episodes 133
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 134
action i 2
action i 3
action i 4
Done 1
N_episodes 135
Done 1
N_episodes 136
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
Done 1
N_episodes 137
Done 1
N_episodes 138
action i 2
action i 3
D

action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
action i 43
action i 44
action i 45
action i 46
action i 47
action i 48
action i 49
action i 50
action i 51
action i 52
action i 53
action i 54
action i 55
action i 56
action i 57
action i 58
action i 59
action i 60
action i 61
action i 62
action i 63
action i 64
action i 65
action i 66
action i 67
action i 68
action i 69
action i 70
action i 71
action i 72
action i 73
action i 74
action i 75
action i 76
action i 77
action i 78
action i 79
action i 80
action i 81
action i 82
action i 83
action i 84
action i 85
action i 86
action i 8

Done 1
N_episodes 255
Done 1
N_episodes 256
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 257
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
Done 1
N_episodes 258
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
Done 1
N_episodes 259
action i 2
Done 1
N_episodes 260
action i 2
action i 3
action i 4
Done 1
N_episodes 261
Done 1
N_episodes 262
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
actio

...

action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
action i 43
action i 44
action i 45
action i 46
action i 47
action i 48
action i 49
action i 50
action i 51
action i 52
action i 53
action i 54
action i 55
action i 56
action i 57
action i 58
Done 1
N_episodes 4591
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4592
action i 2
action i 3
Done 1
N_episodes 4593
Done 1
N_episodes 4594
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
a

action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4676
Done 1
N_episodes 4677
Done 1
N_episodes 4678
action i 2
action i 3
action i 4
Done 1
N_episodes 4679
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
Done 1
N_episodes 4680
Done 1
N_episodes 4681
action i 2
action i 3
Done 1
N_episodes 4682
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
Done 1
N_episodes 4683
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
Done 1
N_episodes 4684
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1

N_episodes 4763
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
Done 1
N_episodes 4764
Done 1
N_episodes 4765
action i 2
Done 1
N_episodes 4766
Done 1
N_episodes 4767
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
Done 1
N_episodes 4768
Done 1
N_episodes 4769
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 4770
action i 2
action i 3
Done 1
N_episodes 4771
action i 2
Done 1
N_episodes 4772
action i 2
action i 3
Done 1
N_episodes 4773
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 4774
Done 1
N_episodes 4775
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4776
Done 1
N_episodes 4777
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 4778
action i 2
action i 3
action i 4
Done 1
N_episodes 4779
action i 2
Done 1
N_episodes 4780
action i 2
Done 

action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
Done 1
N_episodes 4882
Done 1
N_episodes 4883
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 4884
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
Done 1
N_episodes 4885
action i 2
Done 1
N_episodes 4886
Done 1
N_episodes 4887
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
Done 1
N_episodes 4888
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
Done 1
N_episodes 4889
action i 2
action i 3
action i 4
Done 1
N_episodes 4890
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action

action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
Done 1
N_episodes 4993
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
Done 1
N_episodes 4994
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4995
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
Done 1
N_episodes 4996
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
Done 1
N_episodes 4997
Done 1
N_ep

In [11]:
print('Sarsa_Agent, Average reward:',np.mean(R_save),'Number of steps: ',np.mean(N_moves_save))


Random_Agent, Average reward: 0.5708 Number of steps:  7.299


**Initialize Adam Params**

In [10]:
# Let's create a copy of the parameters to be optimised using Adam
W1_A = np.copy(W1)
W2_A = np.copy(W2)
bias_W1_A = np.copy(bias_W1)
bias_W2_A = np.copy(bias_W2)

# Intialise Adam for the parameters
Adam_W1=Adam(W1_A,beta1)
Adam_W2=Adam(W2_A,beta1)
Adam_bias_W1=Adam(bias_W1_A,beta1)
Adam_bias_W2=Adam(bias_W2_A,beta1)

**Copied upper block and Added Adam**

In [12]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...

for n in range(N_episodes):

    epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
    Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
    i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS
    
    S,X,allowed_a=env.Initialise_game()      ## INITIALISE GAME
    print("N_episodes",n)                    ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING
    
    
    while Done==0:                           ## START THE EPISODE
        
        
        ## THIS IS A RANDOM AGENT, CHANGE IT to SARSA with value function approximation and ADAM
        
        #a,_=np.where(allowed_a==1)
        #a_agent=np.random.permutation(a)[0]
        h1 = np.dot(W1_A,X)+bias_W1_A

        # Apply the sigmoid function
        x1 = 1/(1+np.exp(-h1))
        
        Qvalues=np.dot(W2_A,x1)+bias_W2_A
        
        #print("Qvalues",np.shape(Qvalues),Qvalues)
                
        #S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
        
        a_agent = EpsilonGreedy_Policy(Qvalues, epsilon_f)
        S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent)
       # print("S",S,"action",a_agent,R,Done)
        
        ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
        if Done==1:
            print("Done",Done)
            # Compute the delta
            #delta=R-Qvalues[a]
            
            # Update the Qvalues
            delta=R-Qvalues[a_agent]
                
            aa = eta*Adam_W2.Compute(delta)*x1.T
            
            W2_A[a_agent,:]=W2_A[a_agent,:]+aa[a_agent]
            bias_W2_A[a_agent] = bias_W2_A[a_agent]+eta*Adam_bias_W2.Compute(delta)[a_agent]
            
            W1_A=W1_A+eta*Adam_W1.Compute(delta)*X.T
            bias_W1_A = bias_W1_A+eta*Adam_bias_W1.Compute(delta)
            #b1[a_agent,:]=b1[a_agent,:]+eta*delta*X
            #b2=b2+eta*delta*h1
            #save the R_save,N_moves_save
            R_save[n]=np.copy(R)
            N_moves_save[n]=np.copy(i)
            
            
            break
        
        # IF THE EPISODE IS NOT OVER...
        h2 = np.dot(W1_A,X_next)+bias_W1_A

        # Apply the sigmoid function
        x2 = 1/(1+np.exp(-h2))
        
        Qvalues2=np.dot(W2_A,x2)+bias_W2_A
        a_pri = EpsilonGreedy_Policy(Qvalues2, epsilon_f)
        
        # Compute the delta
        delta=R+gamma*Qvalues2[a_pri]-Qvalues[a_agent]
        
        # Update the weights
        #print(np.shape(W2[a_agent,:]),np.shape(delta),np.shape(Qvalues2[a_pri]))
        bb = eta*Adam_W2.Compute(delta)*x1.T
        W2_A[a_agent,:]=W2_A[a_agent,:]+bb[a_agent]
        bias_W2_A[a_agent] = bias_W2_A[a_agent]+eta*Adam_bias_W2.Compute(delta)[a_agent]
            
        W1_A=W1_A+eta*Adam_W1.Compute(delta)*X.T
        bias_W1_A = bias_W1_A+eta*Adam_bias_W1.Compute(delta)
            
        # NEXT STATE AND CO. BECOME ACTUAL STATE...     
        S=np.copy(S_next)
        X=np.copy(X_next)
        allowed_a=np.copy(allowed_a_next)
        
        i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
        print("action i",i)
        

N_episodes 0
action i 2
Done 1
N_episodes 1
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 2
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
Done 1
N_episodes 3
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 4
action i 2
action i 3
Done 1
N_episodes 5
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 6
action i 2
action i 3
action i 4
Done 1
N_episodes 7
Done 1
N_episodes 8
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 9
action i 2
Done 1
N_episodes 10
action i 2
Done 1
N_episodes 11
Done 1
N_episodes 12
Don

N_episodes 118
action i 2
Done 1
N_episodes 119
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 120
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
Done 1
N_episodes 121
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
Done 1
N_episodes 122
Done 1
N_episodes 123
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 124
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
Done 1
N_episodes 125
action i 2
action i 3
action i 4
Done 1
N_episodes 126
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
Done 1
N_episodes 127
Done 1
N_episodes 128
action i 2
action i 3
action i 4
action i 

action i 3
action i 4
action i 5
Done 1
N_episodes 256
action i 2
Done 1
N_episodes 257
Done 1
N_episodes 258
action i 2
Done 1
N_episodes 259
action i 2
action i 3
action i 4
Done 1
N_episodes 260
action i 2
Done 1
N_episodes 261
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
Done 1
N_episodes 262
Done 1
N_episodes 263
action i 2
action i 3
action i 4
Done 1
N_episodes 264
Done 1
N_episodes 265
Done 1
N_episodes 266
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 267
Done 1
N_episodes 268
Done 1
N_episodes 269
Done 1
N_episodes 270
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
Done 1
N_episodes 271
action i 2


action i 22
Done 1
N_episodes 368
action i 2
Done 1
N_episodes 369
Done 1
N_episodes 370
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
action i 43
action i 44
action i 45
action i 46
action i 47
action i 48
action i 49
action i 50
action i 51
action i 52
action i 53
action i 54
action i 55
action i 56
action i 57
action i 58
action i 59
action i 60
action i 61
action i 62
action i 63
action i 64
action i 65
action i 66
action i 67
action i 68
Done 1
N_episodes 371
action i 2
action i 3
action i 4
Done 1
N_episodes 372
action i 2
action i 3
Done 1
N_episode

action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
action i 43
action i 44
action i 45
action i 46
action i 47
action i 48
action i 49
action i 50
action i 51
action i 52
action i 53
action i 54
action i 55
action i 56
action i 57
action i 58
action i 59
action i 60
action i 61
action i 62
action i 63
action i 64
action i 65
action i 66
action i 67
action i 68
action i 69
action i 70
action i 71
action i 72
action i 73
action i 74
action i 75
action i 76
action i 77
Done 1
N_episodes 459
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 460
Done 1
N_episodes 461
action i 2
Done 1
N_episodes 462
Done 1
N_episodes 463
action i 2
action i 3
Done 1
N_episodes 464
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
Done 1
N_episodes 465
action i

...

N_episodes 4500
action i 2
Done 1
N_episodes 4501
action i 2
Done 1
N_episodes 4502
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
action i 43
action i 44
Done 1
N_episodes 4503
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4504
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4505
Done 1
N_episodes 4506
Done 1
N_episodes 4507
action i 2
Done 1
N_episodes 4508
Done 1
N_episodes 4509
action i 2
Done 1
N_episodes 4510
action i 2
Done 1
N_episodes 4511
Done 1
N_episo

N_episodes 4599
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 4600
action i 2
action i 3
Done 1
N_episodes 4601
action i 2
Done 1
N_episodes 4602
action i 2
action i 3
action i 4
Done 1
N_episodes 4603
Done 1
N_episodes 4604
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
Done 1
N_episodes 4605
action i 2
Done 1
N_episodes 4606
Done 1
N_episodes 4607
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
Done 1
N_episodes 4608
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i

action i 2
action i 3
Done 1
N_episodes 4688
Done 1
N_episodes 4689
action i 2
action i 3
Done 1
N_episodes 4690
Done 1
N_episodes 4691
action i 2
action i 3
action i 4
action i 5
action i 6
Done 1
N_episodes 4692
action i 2
Done 1
N_episodes 4693
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
Done 1
N_episodes 4694
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
Done 1
N_episodes 4695
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 4696
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
Done 1
N_epi

action i 33
action i 34
action i 35
action i 36
action i 37
action i 38
action i 39
action i 40
action i 41
action i 42
action i 43
action i 44
action i 45
action i 46
action i 47
action i 48
action i 49
action i 50
action i 51
action i 52
action i 53
Done 1
N_episodes 4791
Done 1
N_episodes 4792
Done 1
N_episodes 4793
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
Done 1
N_episodes 4794
Done 1
N_episodes 4795
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
Done 1
N_episodes 4796
action i 2
action i 3
action i 4
action i 5
Done 1
N_episodes 4797
action i 2
Done 1
N_episodes 4798
action i 2
Done 1
N_episodes 4799
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
Done 1
N_episodes 4800
action i 2
action i 3
Done 1
N_episodes 4801
action i 2
Done 1
N_episodes 4802
a

action i 11
action i 12
action i 13
action i 14
Done 1
N_episodes 4908
Done 1
N_episodes 4909
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
action i 14
action i 15
action i 16
action i 17
action i 18
action i 19
action i 20
action i 21
action i 22
action i 23
action i 24
action i 25
action i 26
action i 27
action i 28
action i 29
action i 30
action i 31
action i 32
Done 1
N_episodes 4910
Done 1
N_episodes 4911
Done 1
N_episodes 4912
action i 2
Done 1
N_episodes 4913
Done 1
N_episodes 4914
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 13
Done 1
N_episodes 4915
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
Done 1
N_episodes 4916
action i 2
Done 1
N_episodes 4917
action i 2
action i 3
action i 4
action i 5
action i 6
action i 7
action i 8
action i 9
action i 10
action i 11
action i 12
action i 1

In [13]:
print('Adam_Sarsa, Average reward:',np.mean(R_save),'Number of steps: ',np.mean(N_moves_save))

Random_Agent, Average reward: 0.4952 Number of steps:  7.2008
